In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from PIL import Image, ImageEnhance
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 2) Veri Setini Hazırlama

### Veri setinin dosya yapısını inceleme
1) **Base Directory Kontrolü:** Veri setinin bulunduğu ana klasörü listeledim ve içinde hangi alt klasör olduğunu kontrol ettim.Bu, veri setine doğru şekilde eriştiğimi doğrulamak için önemli bir adım.
2) **Alt Klasörlerin ve Dosyaların Listelenmesi:** Ana klasör içindeki alt klasörü ('Animals_with_Attributes2') listeleyerek veri setinin yapısını anlamaya çalıştım. Doğru klasöre erişim sağladığımdan emin oldum.
3) **JPEGImages Klasöründeki Hayvan Sınıflarını Listeleme:** Hayvan görüntülerinin yer aldığı `JPEGImages` klasörünü listeledim ve içindeki sınıfları inceledim.Bu, sınıflandırma yapacağım veri setini anlamama yardımcı oldu.

In [ ]:
# Base Directory Kontrolü
base_dir = '/kaggle/input/animals-with-attributes-2'
print(os.listdir(base_dir))  # Bu klasörde hangi dosyalar var?

In [ ]:
# Alt Klasörlerin ve Dosyaların Listelenmesi
sub_dir = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2'
print(os.listdir(sub_dir))  # Alt klasörleri ve dosyaları listeleme

In [ ]:
# JPEGImages Klasöründeki Hayvan Sınıflarını Listeleme
image_dir = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'
print(os.listdir(image_dir))  # Hayvan sınıflarını listeleme

### Gerekli Sınıfları Kontrol Etme
Projede dosyasında belirtilen ve kullanılacak olan 10 sınıfın 'JPEGImages' klasöründe bulunup bulunmadığını kontrol ettim. Bu adım, veri setindeki doğru sınıflarla çalışmaya başlayabilmek için önemli.

In [ ]:
# Kullanılacak sınıflar
selected_classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant+panda', 'polar+bear']

for class_name in selected_classes:
    if class_name in os.listdir(image_dir):
        print(f"{class_name} klasörü bulundu.")
    else:
        print(f"{class_name} klasörü bulunamadı!")

### Gerekli görüntüleri seç ve yeni klasöre  kopyala
Projede kullanılacak olan 10 sınıf için her sınıftan sadece ilk 650 görüntüyü seçip yeni bir klasöre (*filtered_dir*) kopyaladım ve sonrasında yeni klasördeki sınıfları ve görüntü sayılarını kontrol ettim. Veri setindeki sınıfların dengeli olmasını sağlamak için önemli bir adım.

In [130]:
# Yeni bir hedef klasör oluşturuldu
filtered_dir = '/kaggle/working/filtered_data'
os.makedirs(filtered_dir, exist_ok=True)

for class_name in selected_classes:
    source_class_dir = os.path.join(image_dir, class_name) # Orjinal sınıf klasörü
    target_class_dir = os.path.join(filtered_dir, class_name) # Yeni hedef klasör
    os.makedirs(target_class_dir, exist_ok = True) # Hedef klasör oluşturma

    # Her sınıf için ilk 650 görüntüyü seç ve yeni klasöre kopyala
    images = os.listdir(source_class_dir)[:650]
    for image_name in images:
        shutil.copy(os.path.join(source_class_dir, image_name), target_class_dir)

In [ ]:
# Yeni klasördeki sınıfları ve görüntü sayısını kontrol etme
for class_name in selected_classes:
    class_dir = os.path.join(filtered_dir, class_name)
    print(f"{class_name}: {len(os.listdir(class_dir))} images")

### Tüm görüntüleri aynı boyuta getir ve normalize et

In [132]:
def preprocess_image(image_path, size = (128,128)):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image,size)
    image_normalized = image_resized / 255.0
    return image_normalized

In [133]:
X = []  # Görüntü veri seti
y = []  # Etiketler

for idx, class_name in enumerate(selected_classes):
    class_dir = os.path.join(filtered_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        processed_image = preprocess_image(image_path)
        X.append(processed_image)
        y.append(idx)

In [ ]:
# Listeleri numpy dizilerine dönüştür
X = np.array(X)
y = np.array(y)

# İşlemlerin sonuçlarını kontrol et
print(f"Toplam görüntü sayısı: {len(X)}")
print(f"Görüntü boyutları: {X[0].shape}")
print(f"Toplam etiket sayısı: {len(y)}")

#### Veri İşleme Sonuçları 

Aşağıdaki sonuçlar, görüntülerin işlenip boyutlandırılması ve etiketlenmesi sonrasında elde edilmiştir:

1. Toplam görüntü sayısı: **6500**
- Her sınıf için(10 sınıf) **650 görüntü** işlenmiştir. (10 x 650= 6500)
- Modelin her sınıfa eşit derecede öğrenme şansı olması sağlandı

2. Görüntü boyutları: **(128,128,3)**
- Tüm görüntüler 128x128 piksel boyutunda yeniden boyutlandırılmıştır.
- *3*: Görüntülerin renkli olduğunu ifade eder. (RGB renk kanalları: kırmızı, yeşil, mavi)

3. Toplam etiket sayısı: **6500**
- Her görüntüye bir sınıf etiketi atanmıştır.
- Etiketlerin sayısı ile toplam görüntü sayısı aynıdır. Bunun anlamı, veri işleme sırasında herhangi bir eksik görüntü veya etiket olmadığını doğrular. Eğitim aşamasına geçmeye hazır olduğumu göstermektedir.

### Veriyi train(eğitim) ve test setlerine ayırma
Veri setini %70 eğitim ve %30 test olacak şekilde ikiye böldüm. Bu işlem, modelin eğitim ve değerlendirme için ayrı veri setlerinde çalışmasını sağlar.

In [ ]:
# Görüntüleri eğitim ve test setine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Sonuçları yazdır
print(f"Eğitim seti boyutu: {len(X_train)} görüntü")
print(f"Test seti boyutu: {len(X_test)} görüntü")
print(f"Eğitim etiketleri: {len(y_train)}")
print(f"Test etiketleri: {len(y_test)}")

### Eğitim setine veri arttırma(Augmentation) uygulama

#### **1. Bulanıklaştırma**

In [136]:
def blur_image(image):
    return cv2.GaussianBlur(image, (5,5), 0)

#### **2. Kenar Bulma**

In [137]:
def edge_detection(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY) # siyah-beyaz çevirme
    edges = cv2.Canny(gray, 100, 200) # kenar bulma
    return edges

#### **3.Yeniden Boyutlandırma**

In [138]:
def resize_image(image, size= (64,64)):
    return cv2.resize(image, size)

#### **4. Döndürme**

In [139]:
# Görüntüyü 30 derece döndürme
def rotate_image(image, angle = 30):
    (h,w) = image.shape[:2] # görüntünün ilk 2 boyutunu alma(yükseklik ve genişlik)
    center = ( w // 2, h // 2) # görüntünün merkezini hesaplama
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w,h)) # görüntüyü döndürmek için hesaplanan matrisi uygula
    return rotated

#### **5. Gürültü Ekleme**

In [140]:
def add_noise(image):
    noise = np.random.normal(0, 0.05, image.shape)
    noisy_image = np.clip(image + noise, 0, 1) # piksel değerleri 0-1 arasında sınırlandırıldı(0dan küçükse 0, 1den büyükse 1)
    return noisy_image

#### **Tüm Manipülasyon Teknikleri Eğitim Setine Uygulama**

In [ ]:
augmented_images = [] # manipüle edilmiş görüntülerin saklanacağı liste

# Tüm eğitim setine manipülasyon uygula
for idx in range(len(X_train)): # tüm eğitim seti
    
    original_image = X_train[idx]

    # manipülasyon işlemleri
    blurred = blur_image(original_image)
    edges = edge_detection(original_image)
    resized = resize_image(original_image)
    rotated = rotate_image(original_image)
    noisy = add_noise(original_image)

    # manipülasyon işlemlerini kaydet
    augmented_images.append({
        "original": original_image,
        "blurred": blurred,
        "edges": edges,
        "resized": resized,
        "rotated": rotated,
        "noisy": noisy
    })
    
print("Tüm eğitim manipülasyonu işlemi başarıyla tamamlandı!")

In [ ]:
result = augmented_images[0] # listenin ilk elemanını al

fig, axs = plt.subplots(1, 6, figsize = (20,5)) # 1 satır, 6 sütundan oluşan ve her sütun bir manipüle edilmiş görüntüyü temsil eder
axs[0].imshow(result["original"]) # ilk manipüle edilmiş görüntünün orjinal halini göster
axs[0].set_title("Original")

axs[1].imshow(result["blurred"])
axs[1].set_title("Bulanık")

axs[2].imshow(result["edges"], cmap="gray") 
axs[2].set_title("Kenar Bulma")

axs[3].imshow(result["resized"])
axs[3].set_title("Yeniden Boyutlandır")

axs[4].imshow(result["rotated"])
axs[4].set_title("Döndürme")

axs[5].imshow(result["noisy"])
axs[5].set_title("Gürültü")


In [ ]:
result1 = augmented_images[0] # listenin ilk elemanını al

fig, axs = plt.subplots(1, 6, figsize = (20,5)) # 1 satır, 6 sütundan oluşan ve her sütun bir manipüle edilmiş görüntüyü temsil eder
axs[0].imshow(result["original"]) # ilk manipüle edilmiş görüntünün orjinal halini göster
axs[0].set_title("Original")

axs[1].imshow(result["blurred"])
axs[1].set_title("Bulanık")

axs[2].imshow(result["edges"], cmap="gray") 
axs[2].set_title("Kenar Bulma")

axs[3].imshow(result["resized"])
axs[3].set_title("Yeniden Boyutlandır")

axs[4].imshow(result["rotated"])
axs[4].set_title("Döndürme")

axs[5].imshow(result["noisy"])
axs[5].set_title("Gürültü") 

for ax in axs:
    ax.axis("off")
plt.tight_layout()
plt.show()

## **3) CNN Modeli Tasarlama**

Bu model, hem düşük seviyeli hem de yüksek seviyeli özellikleri öğrenerek 10 sınıfa ait hayvan görüntülerini doğru bir şekilde sınıflandırmayı hedeflemektedir. Dropout kullanılarak aşırı öğrenme engellenmiş, softmax aktivasyonu ile sınıf olasılıkları hesaplanmıştır.

### **Model Katmanları**
1. **Conv2D Katmanları (Konvolüsyon):***
- Görüntüden özellikleri çıkarmak için kullanılır.
- Modelde 3 kovülasyon katmanı bulunmaktadır.
  1) İlk katman temel özellikleri (kenarlar, köşeler)
  2) İkinci katman orta seviyeli özellikler
  3) Üçüncü katman ise daha karmaşık, yüksek seviyeli özellikler

2. **MaxPooling2D Katmanları (Havuzlama):**
- Görüntü boyutlarını küçültmek ve modelin hesaplama maliyetini azaltmak için kullanılır.
- Her kovülasyon katmanının ardından bir max pooling katmanı eklenmiştir.

3. **Flatten Katmanı:**
- Konvülasyon ve havuzlama katmanlarının çıktısını düzleştirerek fully connected katmanlara hazırlar.

4. **Dense Katmanları (Tam Bağlantılı):**
- Öğrenilen özellikleri sınıflandırma işlemine dönüştürmek için kullanılır.
- Son katman, *"softmax"* aktivasyon fonksiyonu ile her sınıfa bir olasılık atar.

5. **Dropout:**
- Aşırı öğrenmeyi engellemek için fully connected katmanda kullanılmıştır.

In [ ]:
# CNN Modeli
model = Sequential()

# 1. Kovülasyon Katmanı
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (128, 128, 3))) # 32 filtre, 3x3 çekirdek
model.add(MaxPooling2D(pool_size = (2,2))) # 2x2 boyutunda havuz

# 2. Kovülasyon Katmanı
model.add(Conv2D(64, (3,3), activation = 'relu')) # 64 filtre
model.add(MaxPooling2D(pool_size = (2,2)))

# 3. Kovülasyon Katmanı
model.add(Conv2D(128, (3,3), activation = 'relu')) # 128 filtre
model.add(MaxPooling2D(pool_size = (2,2)))

# Fully Connected Katmanları
model.add(Flatten()) # veriyi düzleştirme
model.add(Dense(128, activation = 'relu')) # 128 nöronlu tam bağlantılı katman
model.add(Dropout(0.5)) # aşırı öğrenmeyi önlemek için
model.add(Dense(10, activation = 'softmax')) # çıkış katmanı

# Modeli Derleme
model.compile(optimizer= Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model özeti
model.summary()

#### **Toplam Parametre Sayısı:**
- **Toplam:** 3,305,930
- **Eğitilebilir:** 3,305,930
- **Eğitilemez:** 0

### **Modelin Çalışma Mantığı**
1. **Giriş:** 128x128 boyutunda RGB görüntüler.
2. **Çıkış:** 10 sınıf için her birine atanmış olasılık değerleri (softmax ile)
3.  **Aşamalı Özellik Öğrenimi:**
   - İlk konvolüsyon katmanları düşük seviyeli özellikleri (örneğin kenarlar),
   - Orta katmanlar daha karmaşık özellikleri,
   - Son katmanlar ise tüm görüntünün genel yapısını öğrenir.
4. **Sınıflandırma:** Fully connected katmanlar, öğrenilen özellikleri 10 sınıfa ayırır.

## **4) Modeli Test Etme**

In [145]:
# Etiketleri one-hot encoding formatına dönüştür
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

In [ ]:
history = model.fit(
    X_train, y_train_encoded,
    validation_data=(X_test, y_test_encoded),
    epochs=10,  # Eğitim döngüleri
    batch_size=32,  # Mini-batch boyutu
    verbose=1  # Eğitim sırasında detaylı çıktı
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Doğruluğu: {test_accuracy*100:.2f}%")

Aşağıda, modelin eğitim sürecinde eğitim ve doğrulama doğruluk değerlerini gösteren bir grafik bulunmaktadır.

In [ ]:
# Eğitim ve doğrulama doğruluğu grafiği
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()

### **Model Performansı ve Değerlendirme**
**1. Eğitim Doğruluğu (Accuracy):** Grafiğe baktığımızda, modelin eğitim doğruluğu düzenli bir şekilde arttığı gözüküyor.
- Bu, modelin eğitim verisi üzerinde her epoch'ta daha iyi performans sergilediğini gösterir.

**2. Doğrulama Doğruluğu (Validation Accuracy):** Modelin doğrulama doğruluğu şu şekilde ilerlemiştir:
- Doğrulama doğruluğu başlangıçta hızlı bir artış göstermiş, ancak yaklaşık 5-6 epoch sonrasında bir plato oluşturmuş ve daha fazla artmamıştır.
- Bu durum, modelin doğrulama seti üzerindeki performansının belirli bir seviyeye ulaştıktan sonra iyileşmediğini göstermektedir.
- Eğitim doğruluğu artmaya devam ederken doğrulama doğruluğu sabit kalmış veya hafifçe dalgalanmıştır.

**3. Eğitim ve Doğrulama Kayıpları (Loss):**
- Eğitim kaybı düzenli bir şekilde azalmıştır.
- Doğrulama kaybı bazı noktalarda artmıştır. Bu artış, doğrulama setindeki performansın eğitim setine göre düştüğünü gösterir. Bu, modelin eğitim verisine fazla uyum sağladığını (**overfitting**) gösterebilir.


**Yukarıdaki çıktılara ve grafiklere bakıldığında elde edilen sonuç:** 

Model, test verisiyle test edilmiştir. *Test doğruluğu %61* seviyesinde elde edilmiştir. Projede belirtilen kriterlere göre, model doğruluğu %50’nin üzerinde olduğu için model sınıfları yeterli bir başarıyla ayırabilmektedir. 

Bu doğrultuda, projede belirtilen talimatlara uygun olarak **bir sonraki adıma geçilmiştir.**

## 5) Resimlerin Farklı Işıklar ile Manipüle Edilmesi

In [149]:
def get_manipulated_images_with_labels(image_paths, labels, save_dir, brightness_values=[0.5, 1.5]):
  
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    saved_paths = []  # Kaydedilen dosya yolları
    manipulated_labels = []  # Genişletilmiş etiketler

    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        if image is None:
            print(f"Resim yüklenemedi: {image_path}")
            continue
        image_name = os.path.basename(image_path).split('.')[0]
        
        for brightness in brightness_values:
            manipulated_image = cv2.convertScaleAbs(image, alpha=brightness, beta=0)
            save_path = os.path.join(save_dir, f"{image_name}_brightness_{brightness}.jpg")
            cv2.imwrite(save_path, manipulated_image)
            saved_paths.append(save_path)
            manipulated_labels.append(label)  # Etiketi genişlet

    return saved_paths, manipulated_labels

In [ ]:
print(os.path.exists('/kaggle/working/filtered_data'))

In [ ]:
filtered_data_files = os.listdir('/kaggle/working/filtered_data')
print(f"Dizindeki dosyalar: {filtered_data_files}")

In [ ]:
test_dir = '/kaggle/working/filtered_data'

# Alt klasörlerdeki tüm görüntü dosyalarını bulun
image_paths = [
    os.path.join(root, file)
    for root, _, files in os.walk(test_dir)
    for file in files
    if file.lower().endswith(('.png', '.jpg', '.jpeg'))
]

print(f"Bulunan test görüntüsü sayısı: {len(image_paths)}")

In [ ]:
save_dir = '/kaggle/working/filtered_data'   # Manipüle edilmiş görüntülerin kaydedileceği klasör
manipulated_paths, manipulated_labels = get_manipulated_images_with_labels(
    image_paths=image_paths,
    labels=y_test_encoded,  # Test setindeki etiketler
    save_dir=save_dir,
    brightness_values=[0.5, 1.5]
)

print(f"Manipüle edilmiş görüntü sayısı: {len(manipulated_paths)}")
print(f"Manipüle edilmiş etiket sayısı: {len(manipulated_labels)}")

In [ ]:
print(os.listdir(save_dir))  # Manipüle edilmiş görüntülerin olduğu dosyaları listele

In [ ]:
# Manipüle edilmiş görüntülerin olduğu dizin
manipulated_dir = '/kaggle/working/filtered_data'

# İlk görüntüyü seç
first_image_name = os.listdir(manipulated_dir)[0]  # İlk görüntü dosyasını seç
first_image_path = os.path.join(manipulated_dir, first_image_name)

# Orijinal ve manipüle edilmiş görüntüleri yükle
original_image = cv2.imread(first_image_path)
manipulated_image = cv2.convertScaleAbs(original_image, alpha=1.5, beta=0)

# Görüntüleri göster
plt.figure(figsize=(10, 5))

# Orijinal görüntü
plt.subplot(1, 2, 1)
plt.title("Orijinal")
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

# Manipüle edilmiş görüntü
plt.subplot(1, 2, 2)
plt.title("Parlaklık Manipülasyonu")
plt.imshow(cv2.cvtColor(manipulated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()

Yukarıdaki görselde, test görüntülerinden birine parlaklık manipülasyonu uygulanmıştır. Soldaki görsel, manipülasyon öncesindeki orjinal görüntüdür Görsel normal parlaklık seviyesinde olup herhanbir bir işlem uygulanmamıştır. Sağdaki görsel, orjinal görüntüye 'alpha= 1.5' parlaklık faktörü kullanılarak manipüle edilmiştir.Görseldeki parlaklık artışı sayesinde detaylar daha belirgin hale gelmiştir. Renkler daha aydınlık bir tondadır.

In [ ]:
# Orijinal test setini model üzerinde değerlendirin
test_loss, original_test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=1)

# Doğruluk oranını yazdırın
print(f"Orijinal test doğruluğu: {original_test_accuracy * 100:.2f}%")

## 6) Modelin Manipüle Edilmiş Test Seti ile Denenmesi

In [ ]:
manipulated_test_images = np.array([preprocess_image(path, size=(128, 128)) for path in manipulated_paths])
manipulated_test_labels = np.array(manipulated_labels)

print(f"Manipüle edilmiş test setinin görüntü boyutu: {manipulated_test_images.shape}")
print(f"Manipüle edilmiş test setinin etiket boyutu: {manipulated_test_labels.shape}")

In [ ]:
# Manipüle edilmiş test setini model üzerinde değerlendir
test_loss, manipulated_test_accuracy = model.evaluate(manipulated_test_images, manipulated_test_labels, verbose=1)
print(f"Manipüle edilmiş test doğruluğu: {manipulated_test_accuracy * 100:.2f}%")

**Manipüle Edilmiş Test Seti Sonucu:**
Manipüle edilmiş test seti üzerinde modelin doğruluğu **%10** seviyesinde ölçülmüştür. Bu, manipüle edilmiş veri üzerindeki düşük doğruluk oranını ve modelin bu tür verilere genelleme yeteneğinin sınırlı olduğunu göstermektedir.Manipüle edilmiş test seti, modelin eğitim sırasında gördüğü veri dağılımından farklıdır. Bu, modelin manipüle edilmiş verilere uyum sağlayamadığını ve sınıflandırma görevini doğru şekilde yerine getiremediğini göstermektedir.


## **7) Manipüle Edilmiş Test Setine Renk Sabitliği Algoritması Uygulanması**

In [159]:
def get_wb_images(image_paths, save_dir):
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    wb_images = []

    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Resim yüklenemedi: {image_path}")
            continue
        image_name = os.path.basename(image_path).split('.')[0]
        
        # Gray World algoritması
        result = cv2.xphoto.createGrayworldWB()
        wb_image = result.balanceWhite(image)

        save_path = os.path.join(save_dir, f"{image_name}_wb.jpg")
        cv2.imwrite(save_path, wb_image)
        wb_images.append(save_path)
        print(f"{save_path} kaydedildi.")

    return wb_images

In [ ]:
# Manipüle edilmiş test setinin dosya yolları
manipulated_test_dir = '/kaggle/working/manipulated_test_images'  # Manipüle edilmiş test seti yolu
save_dir_wb = '/kaggle/working/wb_test_images'  # Kaydedilecek renk sabitliği uygulanmış test seti

# Gray World algoritması ile renk sabitliği uygula
wb_image_paths = get_wb_images(manipulated_paths, save_dir_wb)

# Renk sabitliği uygulanmış görüntü sayısını yazdır
print(f"Renk sabitliği uygulanmış görüntü sayısı: {len(wb_image_paths)}")

In [ ]:
# Renk sabitliği uygulanmış görüntüleri normalize et
wb_test_images = np.array([preprocess_image(path, size=(128, 128)) for path in wb_image_paths])
wb_test_labels = np.array(manipulated_labels)  # Manipüle edilmiş test setinin etiketlerini kullan

# Boyutları kontrol et
print(f"Renk sabitliği uygulanmış test setinin görüntü boyutu: {wb_test_images.shape}")
print(f"Renk sabitliği uygulanmış test setinin etiket boyutu: {wb_test_labels.shape}")

## **8) Modelin Renk Sabitliği Uygulanmış Test Seti ile Denenmesi**

In [ ]:
# Renk sabitliği uygulanmış test setini model üzerinde değerlendir
test_loss, wb_test_accuracy = model.evaluate(wb_test_images, wb_test_labels, verbose=1)
print(f"Renk sabitliği uygulanmış test doğruluğu: {wb_test_accuracy * 100:.2f}%")

## **9) Farklı Test Setlerinin Başarılarını Karşılaştırma ve Raporlama**

In [ ]:
# Doğruluk oranları
accuracies = [original_test_accuracy * 100, manipulated_test_accuracy * 100, wb_test_accuracy * 100]
labels = ["Orijinal Test Seti", "Manipüle Edilmiş Test Seti", "Renk Sabitliği Uygulanmış Test Seti"]

# Grafik oluşturma
plt.figure(figsize=(10, 6))
plt.bar(labels, accuracies, color=['blue', 'orange', 'green'])
plt.ylabel("Doğruluk Oranı (%)")
plt.title("Farklı Test Setlerinin Doğruluk Oranları")
plt.ylim(0, 100)
plt.show()

## **Raporlama ve Analiz**
Yukarıdaki grafikte, üç farklı test setinin doğruluk oranları karşılaştırılmıştır:

1. **Orijinal Test Seti**: Modelin, orijinal test seti üzerindeki doğruluk oranı %61 seviyesinde ölçülmüştür. Model, orijinal test setinde yeterli performans göstermektedir. %50'nin üzerinde bir doğruluk oranı elde edilmiştir. Bu, modelin eğitim süreci ve veri seti üzerindeki performansının tatmin edici olduğunu göstermektedir.
   
2. **Manipüle Edilmiş Test Seti**: Manipüle edilmiş test seti (farklı parlaklık değerleri uygulanan), model doğruluk oranında ciddi bir düşüşe neden olmuştur (%10 seviyesinde). Manipüle edilmiş test seti üzerinde doğruluk oranı oldukça düşüktür. Bu, modelin veri manipülasyonlarına karşı dayanıklı olmadığını ve parlaklık değişiklikleri gibi faktörlerin sınıflandırma doğruluğunu ciddi şekilde azalttığını göstermektedir.

3. **Renk Sabitliği Uygulanmış Test Seti**: Gray World algoritması ile renk sabitliği uygulanmış test setinde de düşük doğruluk oranı elde edilmiştir. Bu, renk sabitliği algoritmasının tek başına model performansını iyileştirmekte yeterli olmadığını göstermektedir.

**Düşük Sonuçların Nedenleri:**
- Modelin karmaşıklığı manipüle edilmiş verilerle başa çıkabilecek seviyede olmayabilir. CNN mimarisi, veri manipülasyonlarına ve parlaklık değişikliklerine karşı daha dayanıklı olacak şekilde geliştirilebilir.
- Model, eğitim verisine fazla uyum sağlamış olabilir. Bu da test verisi üzerindeki performansın düşmesine neden olabilir.

**İyileştirme Önerileri:**
* Eğitim sırasında veri çeşitliliğini artırmak için veri artırma (data augmentation) tekniklerini kullanabiliriz. (Brightness Range,Rotation, Shear, Zoom vb)
* Model karmaşıklığını arttırmak için ekstra katmanlar ekleyebilir, dropout arttırarak overfitting'i azaltabiliriz.

## **Sonuç**
Bu projede, modelin orijinal test setinde başarılı bir performans sergilemesine rağmen manipüle edilmiş ve renk sabitliği uygulanmış test setlerinde düşük sonuçlar elde edilmiştir. Bu durum, modelin veri çeşitliliği ve dayanıklılık konularında iyileştirilmesi gerektiğini göstermektedir. Yukarıdaki öneriler uygulanarak modelin manipülasyonlara karşı performansı artırılabilir.